### Test splitting on chapters

In [12]:
import importlib
import sys
sys.path.append("../../app/backend")  
import runpod_client
import raw_text_processing
import text_processing
import chunks_processing
import messages_templates
from raw_text_processing import process_pdf

# importlib.reload(toc_parser)
# importlib.reload(raw_text_processing)
# importlib.reload(text_processing)
# importlib.reload(chromadb_utils)
# importlib.reload(chunks_processing)
# importlib.reload(messages_templates)

# from raw_text_processing import process_pdf, extract_pages_range, extract_chapters
# from text_processing import text_chunking, chapters_chunking
# from chromadb_utils import initialize_chromadb, initialize_collection, update_collection
# from chunks_processing import query_collection, get_chapter_context
# from messages_templates import chapter_prompt, chapter_prompt_edgecase, book_prompt


In [7]:
# Dictionary mapping example keys to PDF paths
examples = {
    "pdf_path1": "../../data/mcelreath_2020_statistical-rethinking.pdf",
    "pdf_path2": "../../data/Theory of Statistic.pdf",
    "pdf_path3": "../../data/Deep Learning with Python.pdf",
    "pdf_path4": "../../data/Natural_Image_Statistics.pdf",
    "pdf_path5": "../../data/mml-book.pdf"
}

# Dictionary mapping example keys to page ranges to extract content from
content_page_ranges = {
    "pdf_path1": range(5, 8),
    "pdf_path2": range(10, 17),
    "pdf_path3": range(7, 13),
    "pdf_path4": range(4, 13),
    "pdf_path5": range(2, 5),
}

# Select example number
n_example = 3
key = f"pdf_path{n_example}"
path = examples[key]

In [13]:
process_pdf(path)

TypeError: process_pdf() takes 0 positional arguments but 1 was given

In [9]:
text, pages_data, start_chapter = process_pdf(path)
pages_data_corrected = pages_data[start_chapter:]
toc = extract_pages_range(path, content_page_ranges[key])
print(text[:300])
print('\n\n')
print(toc)

NameError: name 'process_pdf' is not defined

### Extract chapters & chunck them

In [189]:
chapters_json = extract_chapters_from_toc(toc)

use prompt optimized for gemma3
[RunPod] Job started: 978bd049-a800-46a2-85d3-f948da0a44ca-e1
[RunPod] Status: IN_QUEUE
[RunPod] Status: IN_QUEUE
[RunPod] Status: IN_QUEUE
[RunPod] Status: IN_QUEUE
[RunPod] Status: IN_QUEUE
[RunPod] Status: IN_PROGRESS
[RunPod] Status: IN_PROGRESS
[RunPod] Status: IN_PROGRESS
[RunPod] Status: IN_PROGRESS
[RunPod] Status: IN_PROGRESS
[RunPod] Status: IN_PROGRESS
[RunPod] Status: COMPLETED


In [ ]:
chapters_json

In [190]:
chapters = extract_chapters(chapters_json, pages_data_corrected)
chapters = chapters_chunking(chapters)

### Random sampling of chapter chunks

In [224]:
chapter_number = 4
n_questions = 5

chapter_context = get_chapter_context(chapters, chapter_number, n_questions)

### Set up Chroma and chunk text

In [200]:
EMBEDDING_MODEL = "all-MiniLM-L6-v2"  
client, embedding_func = initialize_chromadb(EMBEDDING_MODEL)

# Create two collections with different purposes
whole_text_collection = initialize_collection(client, embedding_func, "whole_text_chunks")
update_collection(whole_text_collection, text, max_words=200, min_words=100, overlap_sentences=3)


In [201]:
update_collection(whole_text_collection, text, max_words=200, min_words=100, overlap_sentences=3)

In [231]:
query = 'data preparation and analysis'
query_context = query_collection(whole_text_collection, query=query, nresults=3, context_multiplier=2)

### Create prompts for question generation

In [240]:
out = chapter_prompt(chapter_context, 5)
out2 = chapter_prompt_edgecase(chapter_context, 5)
out3 = book_prompt(query_context, num_questions=3, user_query=query)

In [242]:
questions = run_prompt(out)

[RunPod] Job started: 049a65ff-4017-4857-b55c-34f31ede75d0-e1
[RunPod] Status: IN_QUEUE
[RunPod] Status: IN_QUEUE
[RunPod] Status: IN_QUEUE
[RunPod] Status: IN_QUEUE
[RunPod] Status: IN_PROGRESS
[RunPod] Status: IN_PROGRESS
[RunPod] Status: IN_PROGRESS
[RunPod] Status: IN_PROGRESS
[RunPod] Status: IN_PROGRESS
[RunPod] Status: COMPLETED


In [244]:
print(questions)

```json
[
{"question": "What is the purpose of preprocess_input in the context of visualizing filters?", "answer": "preprocess_input is used to normalize the input image tensor according to the specific requirements of the VGG16 model, ensuring that the model receives input data in the expected format."},
{"question": "What does the 'block5_conv3' layer represent in the VGG16 model, and why is it targeted for Grad-CAM?", "answer": "block5_conv3 is the last convolutional layer in the VGG16 model. It is targeted for Grad-CAM because it's closer to the final classification layer and captures higher-level features, enabling better localization of important image regions."},
{"question": "How does the deprocess_image function transform a tensor into a displayable image?", "answer": "The deprocess_image function performs several steps: it centers the tensor around 0, normalizes it, clips values to [0, 1], scales it, and finally converts it to an 8-bit unsigned integer format (uint8) within t